In [45]:
#Set-up
import os
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import warnings
from IPython.display import clear_output
import nibabel as nib

import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.utils.data as utils
import time
import copy

In [47]:
x = torch.load("/data/data.pt")
y = torch.load("/data/ages.pt").float()
n = x.shape[0]
y = torch.reshape(y,(n,1))
print(x.shape,y.shape)
print(x.dtype,y.dtype)

torch.Size([536, 3, 224, 224]) torch.Size([536, 1])
torch.float32 torch.float32


In [49]:
fcp_x = torch.load("/data/fcp_data.pt")
fcp_y = torch.load("/data/fcp_ages.pt").float()
n = fcp_x.shape[0]
fcp_y = torch.reshape(fcp_y,(n,1))
print(fcp_x.shape,fcp_y.shape)
print(fcp_x.dtype,fcp_y.dtype)

torch.Size([1034, 3, 224, 224]) torch.Size([1034, 1])
torch.float32 torch.float32


In [50]:
adni_x = torch.load('/data/ADNI-CN-data.pt')
adni_y = torch.load('/data/ADNI-CN-data-labels.pt').float()
adni_y = torch.reshape(adni_y, (adni_y.shape[0], 1))
print(adni_x.shape, adni_y.shape)
print(adni_x.dtype, adni_y.dtype)

torch.Size([570, 3, 224, 224]) torch.Size([570, 1])
torch.float32 torch.float32


In [ ]:
#Normalizing the data (according to the stats provided) before feeding into the pretrained model
# fcp_x_normalized = torch.Tensor(fcp_x.shape)
# for i in range(n):
#     fcp_x_normalized[i] = transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                      std=[0.229, 0.224, 0.225]).__call__(fcp_x[i,:,:])


In [51]:
x = torch.cat((x, fcp_x, adni_x))
y = torch.cat((y, fcp_y, adni_y))
print(x.dtype, y.dtype)

torch.float32 torch.float32


In [53]:
USE_GPU = True

dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [54]:
dataset = utils.TensorDataset(x, y)

In [56]:
n = x.shape[0]
print(n)
train_indices = list(range(200, 2000))
val_indices = list(range(2000, 2140))
test_indices = list(range(200))
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=False,sampler=utils.sampler.SubsetRandomSampler(train_indices),num_workers=4)
dataloaders['val'] = torch.utils.data.DataLoader(dataset, batch_size=4,
                                             shuffle=False,sampler=utils.sampler.SubsetRandomSampler(val_indices),num_workers=4)

2140


In [57]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_offset = float('inf')

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_offset = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_offset += torch.sum(torch.abs(outputs - labels.data))

            epoch_loss = running_loss / n
            epoch_offset = running_offset / n

            print('{} Loss: {:.4f} Diff: {:.4f}'.format(
                phase, epoch_loss, epoch_offset))

            # deep copy the model
            if phase == 'val' and epoch_offset < best_offset:
                best_offset = epoch_offset
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Diff: {:4f}'.format(best_offset))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [58]:
model_ft = models.resnet18(pretrained=True)

num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 1)

model_ft = model_ft.to(device)

criterion = nn.MSELoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)

Epoch 0/9
----------
train Loss: 386.6263 Diff: 13.5654
val Loss: 219.5207 Diff: 3.7689

Epoch 1/9
----------
train Loss: 203.2001 Diff: 10.0634
val Loss: 199.8772 Diff: 3.5944

Epoch 2/9
----------
train Loss: 188.9942 Diff: 9.5335
val Loss: 156.2945 Diff: 3.1731

Epoch 3/9
----------
train Loss: 149.3506 Diff: 8.3252
val Loss: 178.5389 Diff: 3.3946

Epoch 4/9
----------
train Loss: 138.3505 Diff: 7.9501
val Loss: 173.6228 Diff: 3.3469

Epoch 5/9
----------
train Loss: 129.0264 Diff: 7.5930
val Loss: 224.9325 Diff: 3.8156

Epoch 6/9
----------


In [42]:
def check_error(data,labels,model):
    print('Checking error on test set')   
    total_abs_err = 0.0
    num_samples = labels.shape[0]
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        data = data.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        labels = labels.to(device=device,dtype=dtype)
        scores = model(data)
        total_abs_err += (torch.abs(labels - scores).sum())
        mean_abs_err = float(total_abs_err) / num_samples
        print('Got error %.2f' % (mean_abs_err))

NameError: name 'tensor' is not defined

In [43]:
check_error(x[test_indices,:], x[test_indices],model_ft)

Checking error on test set


NameError: name 'dtype' is not defined

In [ ]:
# torch.save(model_ft,"the_whole_model.pth")